# Preparing ECC historical data for use in climate record analysis

To format historical weather data going back to the 19th century from Environment and Climate Change Canada so it matches what we get when we call the ECC historical weather API. The idea is that every day, new data will be grabbed using the API, added to the historical data, and then a separate analysis will calculate days since a climate record has been broken.

### Analysis

The data provided by Statscan is a text file that follows the following pattern:

[**Climate ID** (7 digits)] + [**Year** (4 digits)] + [**Month** (2 digits)] + [**Element** (3 digits)] + [**Data for day 1** (6 digits)] ... + [**Data for day 31** (6 digits)]

Element number for Max Temp is 1. Element number for Min Temp is 2.

To split the data up into a CSV, we'll need to isolate each of these sections from approximately 100 separate files. We'll start by importing a few libraries to help.

In [1]:
import pandas as pd
import numpy as np
import glob
import os

pd.options.mode.chained_assignment = None

We'll use glob to iterate through all the raw data files from ECC and put them together into one master csv file.

In [2]:
path = os.path.join("./data/", '*.gz')

files = glob.glob(path)

li = []

for filename in files:
    df = pd.read_csv(filename, header=None)
    df.columns
    li.append(df)
    
raw = pd.concat(li)

Here's what each datapoint looks like.

In [3]:
raw.reset_index().at[0,0]

'1018620199212001000061 000079 000070 000058 000061 000084 000076 000071 000069 000058 000043 000054 000075 000096 000064 000056 000047 000032 000055 000061 000076 000118 000116 000093 000056 000079 000052 000009 -00014 -00013 000005 '

Because this is a massive dataset and this notebook is purely instructional, I'm going to reduce the size of this data dramatically, taking only the first two years worth of monthly data. Otherwise, the ~2.5 gb of data would be sluggish, and for the purposes of this demonstration: unnecessary.

In [37]:
data = raw.iloc[0:1000, :]

Now we'll go through each row of data and split it into various other columns based on the template in the technical documentation (and outlined above). We won't touch the actual data yet - we'll handle that next.

In [38]:
data.loc[:, "Climate ID"] = data.loc[:, 0].str[0:7]
data.loc[:, "Year"] = data.loc[:, 0].str[7:11]
data.loc[:, "Month"] = data.loc[:, 0].str[11:13]
data.loc[:, "element_number"] = data.loc[:, 0].str[13:16]

The data should only contain two element numbers: 001 and 002. Even still, we'll filter for these two just in case.

In [39]:
data = data[data["element_number"].isin(["001", "002"])]

data

,0,Climate ID,Year,Month,element_number
0,1018620199212001000061 000079 000070 000058 00...,1018620,1992,12,001
1,1018620199212002-00009 -00011 -00031 -00032 -0...,1018620,1992,12,002
24,1021830199212001000044 000069 000052 000048 00...,1021830,1992,12,001
25,1021830199212002-00019 000005 -00031 -00029 -0...,1021830,1992,12,002
48,1026270199212001000054 000057 000044 000052 00...,1026270,1992,12,001
...,...,...,...,...,...
930,2100630199212002-00146 -00110 000001 -00220 -0...,2100630,1992,12,002
953,2100636199212001-00132 000027 000010 -00132E-0...,2100636,1992,12,001
954,2100636199212002-00210 -00132 -00140 -00148E-9...,2100636,1992,12,002
977,2100685199212001-00147 000081 000024 -00100 -0...,2100685,1992,12,001


Now, we loop through the remaining parts of the string to pull out data for each day and store it in a new column. We start with the first character for the entry for day 1 data, and loop through, taking chunks of 7 characters until we get to the end of the string.

Note that we can't just split the column on spaces, because that space holds data: it's a data flag, according to the technical docs.

In [40]:
start_char = 16
slice_length = 7
max_days = 31

slice_starts = list(range(start_char, (max_days*slice_length)+start_char, slice_length))

slice_starts[0:5]

[16, 23, 30, 37, 44]

Now we use each slice start to take a slice that's 7 characters long from that point.

In [41]:
for i, start_num in enumerate(slice_starts):
    
    start = start_num
    end = start_num + slice_length
    
    data.loc[:, i+1] = data[0].str[start:end].str[0:6]
    
data

,0,Climate ID,Year,Month,element_number,1,2,3,4,5,...,22,23,24,25,26,27,28,29,30,31
0,1018620199212001000061 000079 000070 000058 00...,1018620,1992,12,001,000061,000079,000070,000058,000061,...,000118,000116,000093,000056,000079,000052,000009,-00014,-00013,000005
1,1018620199212002-00009 -00011 -00031 -00032 -0...,1018620,1992,12,002,-00009,-00011,-00031,-00032,-00040,...,000049,000042,000004,-00001,-00009,-00006,-00042,-00044,-00044,-00047
24,1021830199212001000044 000069 000052 000048 00...,1021830,1992,12,001,000044,000069,000052,000048,000064,...,000087,000096,000070,000047,000051,000030,-00027,-00032,-00028,-00010
25,1021830199212002-00019 000005 -00031 -00029 -0...,1021830,1992,12,002,-00019,000005,-00031,-00029,-00016,...,000019,000022,-00004,-00004,-00009,-00029,-00061,-00052,-00040,-00050
48,1026270199212001000054 000057 000044 000052 00...,1026270,1992,12,001,000054,000057,000044,000052,000054,...,000081,000099,000072,000030,000049,000030,-00009,000003,000004,000012
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
930,2100630199212002-00146 -00110 000001 -00220 -0...,2100630,1992,12,002,-00146,-00110,000001,-00220,-00292,...,-00428,-00331,-00338,-99999,-99999,-99999,-99999,-99999,-99999,-99999
953,2100636199212001-00132 000027 000010 -00132E-0...,2100636,1992,12,001,-00132,000027,000010,-00132,-00190,...,-00230,-00216,-00230,-00274,-00275,-00201,-00103,-00108,-00045,-00034
954,2100636199212002-00210 -00132 -00140 -00148E-9...,2100636,1992,12,002,-00210,-00132,-00140,-00148,-99999,...,-00286,-00255,-00305,-00305,-00319,-00308,-00226,-00153,-00131,-00110
977,2100685199212001-00147 000081 000024 -00100 -0...,2100685,1992,12,001,-00147,000081,000024,-00100,-00197,...,-00230,-00208,-00214,-00270,-00260,-00224,-00099,-00093,-00043,-00025


Each of these data points is a string that's 7 digits long. They also each have a precision of one decimal point (as per the technical documentation provided by ECC). So we need to convert each to a float and place the decimal point.

We'll also replace 999999 values with null, as that's what 999999 means in this context.

Note that we're dividing by 10 here because when we convert the string to a float, it adds a .0 rather than taking the last digit as the decimal place. Dividing by 10 is an easy way to move the decimal point left.

In [42]:
data.loc[:, 1:31] = (data.loc[:, 1:31]
                     .astype(str)
                     .replace("\b0+", "", regex=True)
                     .replace("999999|-99999", np.nan, regex=True)
                     .astype(float, errors='ignore')
                     .astype(float, errors='ignore').divide(10)
                     )

data

,0,Climate ID,Year,Month,element_number,1,2,3,4,5,...,22,23,24,25,26,27,28,29,30,31
0,1018620199212001000061 000079 000070 000058 00...,1018620,1992,12,001,6.1,7.9,7.0,5.8,6.1,...,11.8,11.6,9.3,5.6,7.9,5.2,0.9,-1.4,-1.3,0.5
1,1018620199212002-00009 -00011 -00031 -00032 -0...,1018620,1992,12,002,-0.9,-1.1,-3.1,-3.2,-4.0,...,4.9,4.2,0.4,-0.1,-0.9,-0.6,-4.2,-4.4,-4.4,-4.7
24,1021830199212001000044 000069 000052 000048 00...,1021830,1992,12,001,4.4,6.9,5.2,4.8,6.4,...,8.7,9.6,7.0,4.7,5.1,3.0,-2.7,-3.2,-2.8,-1.0
25,1021830199212002-00019 000005 -00031 -00029 -0...,1021830,1992,12,002,-1.9,0.5,-3.1,-2.9,-1.6,...,1.9,2.2,-0.4,-0.4,-0.9,-2.9,-6.1,-5.2,-4.0,-5.0
48,1026270199212001000054 000057 000044 000052 00...,1026270,1992,12,001,5.4,5.7,4.4,5.2,5.4,...,8.1,9.9,7.2,3.0,4.9,3.0,-0.9,0.3,0.4,1.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
930,2100630199212002-00146 -00110 000001 -00220 -0...,2100630,1992,12,002,-14.6,-11.0,0.1,-22.0,-29.2,...,-42.8,-33.1,-33.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN
953,2100636199212001-00132 000027 000010 -00132E-0...,2100636,1992,12,001,-13.2,2.7,1.0,-13.2,-19.0,...,-23.0,-21.6,-23.0,-27.4,-27.5,-20.1,-10.3,-10.8,-4.5,-3.4
954,2100636199212002-00210 -00132 -00140 -00148E-9...,2100636,1992,12,002,-21.0,-13.2,-14.0,-14.8,NaN,...,-28.6,-25.5,-30.5,-30.5,-31.9,-30.8,-22.6,-15.3,-13.1,-11.0
977,2100685199212001-00147 000081 000024 -00100 -0...,2100685,1992,12,001,-14.7,8.1,2.4,-10.0,-19.7,...,-23.0,-20.8,-21.4,-27.0,-26.0,-22.4,-9.9,-9.3,-4.3,-2.5


We're done parsing out the data from the string, so we can drop this column.

In [43]:
data = data.drop(columns=[0])

We want the data to look similar in structure to the format that we get when we call the ECC historical weather data API, since we'll be calling this API to update the data every day.

In [44]:
# Note that this url takes station ID, not climate ID (which is a separate ID for each weather station).
# I grabbed the station ID for climate ID 1018620 from a separate file, which is 118.
api_url = "https://climate.weather.gc.ca/climate_data/bulk_data_e.html?format=csv&stationID=118&Year=1992&timeframe=2"

example_api_data = pd.read_csv(api_url)
example_api_data = example_api_data.loc[example_api_data["Month"] == 12, :"Max Temp (°C)"]

example_api_data.head()

,Longitude (x),Latitude (y),Station Name,Climate ID,Date/Time,Year,Month,Day,Data Quality,Max Temp (°C)
335,-123.43,48.65,VICTORIA INT'L A,1018620,1992-12-01,1992,12,1,NaN,6.1
336,-123.43,48.65,VICTORIA INT'L A,1018620,1992-12-02,1992,12,2,NaN,7.9
337,-123.43,48.65,VICTORIA INT'L A,1018620,1992-12-03,1992,12,3,NaN,7.0
338,-123.43,48.65,VICTORIA INT'L A,1018620,1992-12-04,1992,12,4,NaN,5.8
339,-123.43,48.65,VICTORIA INT'L A,1018620,1992-12-05,1992,12,5,NaN,6.1


It also looks like our data matches the historical file, which is always reassuring!

In order to get the historical data file we're processing to look similar to the data that comes from the API, we'll each row to be a single day. We'll use `pd.melt()` to achieve this.

In [45]:
data = (data
        .loc[data["element_number"].astype(str).str.replace("00", "").astype(int).isin([1, 2]), :]
        .melt(id_vars=["Climate ID", "Year", "Month", "element_number"], value_vars=list(range(1, 31)))
        .rename(columns={"variable": "Day", "climate_id": "Climate ID"})
        )

data.head()

,Climate ID,Year,Month,element_number,Day,value
0,1018620,1992,12,001,1,6.1
1,1018620,1992,12,002,1,-0.9
2,1021830,1992,12,001,1,4.4
3,1021830,1992,12,002,1,-1.9
4,1026270,1992,12,001,1,5.4


Now we'll ensure that our Year, Month, and Day columns are stored as integers so we can sort them properly.

In [46]:
data.loc[:, ["Year", "Month", "Day"]] = data.loc[:, ["Year", "Month", "Day"]].astype(int)

data = (data
        .sort_values(["Climate ID", "Year", "Month", "Day"])
        )

data.head()

,Climate ID,Year,Month,element_number,Day,value
0,1018620,1992,12,001,1,6.1
1,1018620,1992,12,002,1,-0.9
84,1018620,1992,12,001,2,7.9
85,1018620,1992,12,002,2,-1.1
168,1018620,1992,12,001,3,7.0


Because it's useful to have time/date info in one column (and because I use the date to check if new data needs to be grabbed from the API), let's create a "Date/Time" column to match the one that's fetched through the API call.

In [47]:
data["Date/Time"] = pd.to_datetime(data[["Year", "Month", "Day"]], errors='coerce')

data.head()

,Climate ID,Year,Month,element_number,Day,value,Date/Time
0,1018620,1992,12,001,1,6.1,1992-12-01
1,1018620,1992,12,002,1,-0.9,1992-12-01
84,1018620,1992,12,001,2,7.9,1992-12-02
85,1018620,1992,12,002,2,-1.1,1992-12-02
168,1018620,1992,12,001,3,7.0,1992-12-03


Now, a few things:
* We drop duplicates here to ensure there are no blank rows that will interfere with our pivot...
* We pivot the table so we have separate columns for each element number (max temp and min temp, respectively)...
* We rename the new columns to be more descriptive.
* We reset the index, which is multilevel after the pivot.

In [48]:
data = (data
        .drop_duplicates(subset=["Climate ID", "Date/Time", "Year", "Month", "Day", "element_number"])
        .pivot(columns=["element_number"], index=["Climate ID", "Date/Time", "Year", "Month", "Day"], values="value")
        .rename(columns={"001": "Max Temp (°C)", "002": "Min Temp (°C)"})
        .reset_index()
)
                                  
data.head(3)

element_number,Climate ID,Date/Time,Year,Month,Day,Max Temp (°C),Min Temp (°C)
0,1018620,1992-12-01,1992,12,1,6.1,-0.9
1,1018620,1992-12-02,1992,12,2,7.9,-1.1
2,1018620,1992-12-03,1992,12,3,7.0,-3.1


This data now matches the structure of the data we get from ECC api calls quite closely! We can now save it into a csv file (which I won't do here) and add to it every day using a separate analysis.